In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna

In [4]:
from preprocessing import *
data_path = "data/game.csv"
X_train, X_test, y_train, y_test = prep_all(data_path)

X_train = X_train.values.reshape(-1, 1, 136)  # Reshape to (32520, 1, 136)
X_test = X_test.values.reshape(-1, 1, 136)    # Reshape to (N, 1, 136), where N is the number of test samples

train_data = TensorDataset(torch.tensor(X_train), torch.tensor(y_train.to_numpy()))
test_data = TensorDataset(torch.tensor(X_test), torch.tensor(y_test.to_numpy()))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [5]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.linear(x)
        return out


In [6]:
input_size = X_train.shape[2]  # The number of features in your preprocessed data
output_size = 2  # Win or loss (binary classification)

model = LogisticRegressionModel(input_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [7]:
num_epochs = 10000

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.squeeze(1).float(), labels.long()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/100], Loss: 453910036480.0000
Epoch [2/100], Loss: 807108739072.0000
Epoch [3/100], Loss: 667323334656.0000
Epoch [4/100], Loss: 2726778634240.0000
Epoch [5/100], Loss: 277480865792.0000
Epoch [6/100], Loss: 1008112369664.0000
Epoch [7/100], Loss: 3087975055360.0000
Epoch [8/100], Loss: 4031604588544.0000
Epoch [9/100], Loss: 2228678819840.0000
Epoch [10/100], Loss: 2525601988608.0000
Epoch [11/100], Loss: 571748777984.0000
Epoch [12/100], Loss: 1264720150528.0000
Epoch [13/100], Loss: 789811429376.0000
Epoch [14/100], Loss: 1186938617856.0000
Epoch [15/100], Loss: 2269427531776.0000
Epoch [16/100], Loss: 1229055983616.0000
Epoch [17/100], Loss: 1737986015232.0000
Epoch [18/100], Loss: 2102879191040.0000
Epoch [19/100], Loss: 2761501966336.0000
Epoch [20/100], Loss: 1101391724544.0000
Epoch [21/100], Loss: 2110443225088.0000
Epoch [22/100], Loss: 4222777032704.0000
Epoch [23/100], Loss: 443160756224.0000
Epoch [24/100], Loss: 4309106556928.0000
Epoch [25/100], Loss: 7318713139

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in valid_loader:
        inputs, labels = inputs.squeeze(1).float(), labels.long()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')
